In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
from run_tnews_0shot import *

task_name = "tnews"
train_dataset, dev_dataset, test_dataset = load_dataset(task_name)
# train_dataset = cover_dataset2np(train_dataset)
dev_dataset = cover_dataset2np(dev_dataset)
# test_dataset = cover_dataset2np(test_dataset, is_test=True)
# train_dataset = cover_np2tfds(*train_dataset)
dev_dataset = cover_np2tfds(*dev_dataset)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
from transformers import TFGPT2LMHeadModel, TFGPT2ForSequenceClassification, GPT2Config
model_config = GPT2Config.from_pretrained(configs.MODEL_PATH)

model = TFGPT2LMHeadModel.from_pretrained(configs.MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /data2/Text-Suggestion/models.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [19]:
tokenizer = load_tokenizer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [95]:
text = ["今天乌克兰大获全胜", "没有人比我了解python"]
ids = tokenizer(text, add_special_tokens=False, return_tensors='tf', padding=True)['input_ids']
inputs = ids[:,:-1]
labels = ids[:,1:]
out = model(input_ids=inputs, return_dict=True)

In [96]:
out

TFCausalLMOutputWithPast(loss=None, logits=<tf.Tensor: shape=(2, 9, 32128), dtype=float32, numpy=
array([[[  1.9744302 ,  -6.9969406 , -16.79509   , ..., -16.795397  ,
         -16.795376  , -16.795479  ],
        [  4.267757  ,  -1.6535326 ,  -7.440649  , ...,  -7.441683  ,
          -7.441577  ,  -7.4416475 ],
        [  0.19317281,  -1.6998546 , -12.117039  , ..., -12.117353  ,
         -12.117572  , -12.117444  ],
        ...,
        [  0.5127207 ,  -2.5153713 , -12.370995  , ..., -12.372408  ,
         -12.372951  , -12.372885  ],
        [  0.24259272,  -2.2215304 , -13.333269  , ..., -13.33473   ,
         -13.335034  , -13.334824  ],
        [  0.7589359 ,  -1.4690359 ,  -6.8533745 , ...,  -6.8551755 ,
          -6.855259  ,  -6.8553395 ]],

       [[  1.6760646 ,  -7.140033  , -16.826378  , ..., -16.82666   ,
         -16.826647  , -16.826782  ],
        [  1.1758212 ,  -2.6656957 , -11.32017   , ..., -11.32019   ,
         -11.320143  , -11.320188  ],
        [  1.9833513 , 

In [113]:
def loss_fun(labels, logits):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    return loss_fn(labels, logits)
mask = tf.not_equal(labels, 0)
print("mask", mask)
labels = tf.cast(labels,tf.int32) * tf.cast(mask,tf.int32)
print("labels", labels)
raw_loss = loss_fun(labels,out.logits)
print("raw_loss", raw_loss)
mask = tf.cast(mask,tf.float32)
losses = tf.cast(raw_loss,tf.float32) * mask
print("losses", losses)
# losses = tf.reduce_sum(losses,axis=-1) / tf.reduce_sum(mask,axis=-1)
# print("losses", losses)

mask tf.Tensor(
[[ True  True  True  True  True  True False False False]
 [ True  True  True  True  True  True  True  True  True]], shape=(2, 9), dtype=bool)
labels tf.Tensor(
[[28051  5749 26988 27841 27160 27840     0     0     0]
 [26977 27172 26971 26968 27320 27952 27970  6796  1149]], shape=(2, 9), dtype=int32)
raw_loss tf.Tensor(
[[9.3518257e+00 4.0583386e+00 4.1490216e+00 6.1921062e+00 5.2340332e-02
  1.2677022e-03 1.0334850e+01 7.9349532e+00 7.4530206e+00]
 [4.3851023e+00 5.5624809e+00 1.8138936e+00 6.0646114e+00 1.5625546e-02
  1.1773869e+01 4.4465199e+00 1.3087559e+00 6.4660233e-01]], shape=(2, 9), dtype=float32)
losses tf.Tensor(
[[9.3518257e+00 4.0583386e+00 4.1490216e+00 6.1921062e+00 5.2340332e-02
  1.2677022e-03 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [4.3851023e+00 5.5624809e+00 1.8138936e+00 6.0646114e+00 1.5625546e-02
  1.1773869e+01 4.4465199e+00 1.3087559e+00 6.4660233e-01]], shape=(2, 9), dtype=float32)


In [115]:
tf.reduce_sum(losses,axis=-1), tf.reduce_sum(mask,axis=-1)

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([23.804901, 36.01746 ], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([6., 9.], dtype=float32)>)